# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Strategy" data-toc-modified-id="Strategy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Strategy</a></div><div class="lev2 toc-item"><a href="#Thoughts" data-toc-modified-id="Thoughts-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Thoughts</a></div><div class="lev1 toc-item"><a href="#Sample-Data-Generation" data-toc-modified-id="Sample-Data-Generation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sample Data Generation</a></div><div class="lev1 toc-item"><a href="#Data-Loading,-Validation,-and-Prep" data-toc-modified-id="Data-Loading,-Validation,-and-Prep-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Loading, Validation, and Prep</a></div><div class="lev1 toc-item"><a href="#Linear-Program" data-toc-modified-id="Linear-Program-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Linear Program</a></div><div class="lev1 toc-item"><a href="#Matching-Characterization" data-toc-modified-id="Matching-Characterization-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Matching Characterization</a></div>

In [2]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from ast import literal_eval
from collections import Counter
from itertools import product

data_path = './'

# Strategy
* Match based on priority in the tutee's class list (have a coefficient to determine how much this matters?)
* Also prioritize "harder" classes: determine by # total hours of tutoring requested? (weight by # students requesting hours?)
    * Note that this makes the system somewhat gameable: if I really want a tutor and I request 100 hours, I'm more likely to get one because my class will now be seen as "harder" than others
* Match as many tutees as possible
* If >2 hours requested, try not to assign just 1
* Try to leave no tutors with 0 tutees
* Max # tutees per tutor: 3
* Max # hours/match: 3
* Produce multiple output files (probably by changing the relative weights for the priorities above) so they can be compared


## Thoughts
* Max flow to maximize the number of hours of tutoring total (subject to constraints of hours available/request and classes matching) woul be quite elegant, but I can't think of a way to make it work with the desired things to prioritize (the priority for given students, for harder classes, no tutors with 0 tutees (though I don't think that would be much of an issue), and max 3 tutees per tutor; also might yield multiple tutors for a person for the same class (though the below could be used instead to help with that).
    * Could easily set hours requested to one for each student so that this would maximize the number of students matched; then expand the hours used on tutoring matches if tutors have time remaining and more was actually requested.

# Sample Data Generation

In [2]:
def make_sample_files(n, path=data_path):
    next_id = 0
    n_tutees = n
    n_tutors = int(0.6 * n)

    classes = [6.01, 6.001, 6.005, 6.006, 6.036, 6.034, 6.046, 6.047, 6.802, 6.004, 6.002, 6.878, 6.041, 6.042]
    classes = [str(class_) for class_ in classes]
    class_probs = np.array([1., 1, 2, 2, 3, 2, 4, 2, 1, 2, 1, 1, 1, 2])
    class_probs /= class_probs.sum()
    class_ids = {classes[i]: i for i in xrange(len(classes))}
    assert len(class_probs) == len(classes)
    
    ## TUTORS
    with open(path + 'tutor_info.txt', 'w') as tutor_file:
        tutor_file.write('\t'.join(('id', 'name', 'classes', 'avail_hours', 'n_matches')) + '\n')
        for _ in xrange(n_tutors):
            name = reduce(lambda x, y: x+y, np.random.choice(list('aaabcdeeefghiiijklmnooopqrsssttuuuvwxyyz'), size=np.random.randint(3, 10)))
            tid = next_id = next_id + 1
            avail_hours = np.random.choice([0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4])
            n_matches = np.random.choice([0, 0, 0, 0, 1, 1, 2, 3])
            class_list = np.random.choice(classes, p=class_probs, replace=False,
                                          size=np.random.choice([0, 1, 2, 3, 4, 5], p=[.01, .24, .25, .25, .15, .1]))
            class_list = [(class_ids[class_], class_) for class_ in class_list]

            tutor_file.write("\t".join((str(elem) for elem in (tid, name, class_list, avail_hours, n_matches))) + '\n')
    
    ## TUTEES
    with open(path + 'tutee_info.txt', 'w') as tutee_file:
        tutee_file.write('\t'.join(('id', 'name', 'classes', 'n_matches')) + '\n')
        for _ in xrange(n_tutors):
            name = reduce(lambda x, y: x+y, np.random.choice(list('aaabcdeeefghiiijklmnooopqrsssttuuuvwxyyz'), size=np.random.randint(3, 10)))
            tid = next_id = next_id + 1
            n_matches = np.random.choice([0, 0, 0, 0, 1, 1, 2])
            class_list = np.random.choice(classes, p=class_probs, replace=False,
                                          size=np.random.choice([0, 1, 2, 3, 4, 5], p=[.01, .24, .25, .25, .15, .1]))
            class_list = [(class_ids[class_], # class id
                           class_, # class name
                           np.random.choice([1, 2, 2, 2, 3, 3, 4, 5]), # hours requested
                           np.random.choice([0, 0, 0, 1, 1, 2, 3])) # priority: small number -> low priority
                          for class_ in class_list]

            tutee_file.write("\t".join((str(elem) for elem in (tid, name, class_list, n_matches))) + '\n')

In [3]:
make_sample_files(30)
! cat $data_path/tutor_info.txt

id	name	classes	avail_hours	n_matches
1	uxi	[(6, '6.046'), (1, '6.001'), (2, '6.005')]	2	0
2	ythabdbo	[(2, '6.005'), (6, '6.046')]	3	0
3	vivizvim	[(7, '6.047'), (13, '6.042')]	1	1
4	opr	[(3, '6.006'), (6, '6.046'), (8, '6.802'), (11, '6.878')]	3	1
5	krhohuk	[(5, '6.034'), (1, '6.001')]	2	3
6	awwugiu	[(4, '6.036'), (9, '6.004')]	1	0
7	jmeumvw	[(12, '6.041'), (6, '6.046'), (7, '6.047')]	2	0
8	seri	[(6, '6.046')]	1	1
9	iya	[(4, '6.036'), (5, '6.034'), (9, '6.004')]	1	0
10	iuyszae	[(5, '6.034'), (3, '6.006')]	2	0
11	ahnq	[(3, '6.006'), (8, '6.802')]	1	2
12	xjkdzlsws	[(6, '6.046'), (8, '6.802'), (4, '6.036')]	0	3
13	tvdaqi	[(0, '6.01')]	1	0
14	toa	[(5, '6.034'), (6, '6.046')]	1	0
15	sqspbra	[(3, '6.006'), (8, '6.802'), (9, '6.004'), (7, '6.047')]	1	0
16	uvyptsnni	[(6, '6.046'), (13, '6.042'), (5, '6.034'), (3, '6.006'), (9, '6.004')]	4	1
17	xbqbql	[(4, '6.036'), (10, '6.002'), (3, '6.006'), (5, '6.034'), (6, '6.046')]	1	2
18	gau	[(4, '6.036')]	4	1


# Data Loading, Validation, and Prep

In [4]:
def load_tutor_info(verbose=True):
    tutor_info = pd.read_csv(data_path + 'tutor_info.txt', sep='\t', index_col=0).sort_index()
    tutor_info.classes = tutor_info.classes.apply(literal_eval)

    n_zero_hours = (tutor_info.avail_hours == 0).sum()
    if n_zero_hours > 0:
        if verbose:
            print("{} tutors had 0 hours available and are thus being dropped.".format(n_zero_hours))
        tutor_info.drop(tutor_info[tutor_info.avail_hours == 0].index, inplace=True)

    max_matches = 3
    n_max_matches = (tutor_info.n_matches >= max_matches).sum()
    if n_max_matches > 0:
        if verbose:
            print("{} tutors had {} matches already and are thus being dropped.".format(n_max_matches, max_matches))
        tutor_info.drop(tutor_info[tutor_info.n_matches == max_matches].index, inplace=True)

    n_no_classes = (tutor_info.classes.apply(len) == 0).sum()
    if n_no_classes > 0:
        if verbose:
            print("{} tutors had an empty class list and are thus being dropped.".format(n_no_classes))
        tutor_info.drop(tutor_info[tutor_info.classes.apply(len) == 0].index, inplace=True)
    return tutor_info


def load_tutee_info(verbose=True):
    tutee_info = pd.read_csv(data_path + 'tutee_info.txt', sep='\t', index_col=0).sort_index()
    tutee_info.classes = tutee_info.classes.apply(literal_eval)

    n_no_classes = (tutee_info.classes.apply(len) == 0).sum()
    if n_no_classes > 0:
        if verbose:
            print("{} tutees had an empty class list and are thus being dropped.".format(n_no_classes))
        tutee_info.drop(tutee_info[tutee_info.classes.apply(len) == 0].index, inplace=True)

    return tutee_info

In [5]:
tutor_info = load_tutor_info()
n_tutors = len(tutor_info)
tutor_to_idx = {tutor_info.index.values[i]: i for i in xrange(n_tutors)}
idx_to_tutor = {val: key for (key, val) in tutor_to_idx.items()}
tutor_info.head()

1 tutors had 0 hours available and are thus being dropped.
1 tutors had 3 matches already and are thus being dropped.


,name,classes,avail_hours,n_matches
id,,,,
1,uxi,"[(6, 6.046), (1, 6.001), (2, 6.005)]",2,0
2,ythabdbo,"[(2, 6.005), (6, 6.046)]",3,0
3,vivizvim,"[(7, 6.047), (13, 6.042)]",1,1
4,opr,"[(3, 6.006), (6, 6.046), (8, 6.802), (11, 6.878)]",3,1
6,awwugiu,"[(4, 6.036), (9, 6.004)]",1,0


In [6]:
tutee_info = load_tutee_info()
n_tutees = len(tutee_info)
tutee_to_idx = {tutee_info.index.values[i]: i for i in xrange(n_tutees)}
idx_to_tutee = {val: key for (key, val) in tutee_to_idx.items()}
tutee_info.head()

,name,classes,n_matches
id,,,
19,aueps,"[(6, 6.046, 3, 1), (5, 6.034, 1, 2), (3, 6.006...",0
20,aauz,"[(12, 6.041, 5, 3), (9, 6.004, 2, 0), (3, 6.00...",0
21,iiaelr,"[(12, 6.041, 2, 0)]",1
22,aweeg,"[(6, 6.046, 3, 0), (4, 6.036, 2, 0), (3, 6.006...",1
23,aqotvyey,"[(1, 6.001, 3, 0), (9, 6.004, 5, 0)]",2


In [7]:
class_id_name = np.concatenate((tutee_info.classes.map(lambda class_list: [class_elem[:2] for class_elem in class_list]).values,
                tutor_info.classes.values))
class_to_id = {name: idx for (idx, name) in reduce(lambda x, y: x + y, class_id_name)}

# the lists in classes have elements (class_id, class_name, hours_requested, priority)
# find the total number of hours requested per class by making a dict: {name: hours} for each row, then convert
# to a Counter so that you can sum them all together into one

hours_per_class_list = tutee_info.classes.apply(lambda x: {elem[1]: elem[2] for elem in x}).values
class_priority = sum((Counter(d) for d in hours_per_class_list), Counter())

for class_ in class_to_id:
    if class_ not in class_priority:
        class_priority[class_] = 0 # no tutees requested this class, though there are tutors available

class_priority = pd.DataFrame(class_priority.items(), columns=['class_name', 'priority']).sort_values('class_name')
class_priority.priority /= class_priority.priority.sum()


n_classes = len(class_priority)
class_to_idx = {class_priority.class_name.values[i]: i for i in xrange(n_classes)}
idx_to_class = {val: key for (key, val) in class_to_idx.items()}
get_class_idx = np.vectorize(class_to_idx.get)
class_priority.head()

,class_name,priority
5,6.001,0.047619
6,6.002,0.031746
8,6.004,0.206349
7,6.005,0.071429
9,6.006,0.063492


# Linear Program

This part is tricky. What we really have is a 3D tensor of shape (n_tutors, n_tutees, n_classes). The value at each cell $(i,j,k)$ of the tensor is the number of hours that tutor $T_i$ is tutoring tutee (student, hereafter) $S_j$ in class $C_k$. However, we want to represent each element of this tensor as a variable so that we can optimize the tutoring with a linear program. So, we ravel the tensor into one dimension. We just have to be careful to keep the proper order of things. We'll mentally ravel the 3D tensor into 1D in reading order: top right to bottom left, front to back. This means that students change the most quickly, then tutors, then classes. As an example:
$$(T_0,S_0,C_0), (T_0,S_1,C_0),...,(T_1,S_0,C_0),(T_1,S_1,C_0),...,(T_0,S_0,C_1),(T_0,S_1,C_1),...$$
We'll refer to the 1D array as $A$ (creative). If there are $n_t$ teachers, $n_s$ students, and $n_c$ classes, then we index it like:
$$(T_i,S_j,C_k)=A_{j+n_s*i+(n_s*n_t)*c}$$
To make this simpler to write, we may denote it $A_{i,j,k}$ at times, but the above is what is meant by that.

**Objective function**

We'll optimize a weighted sum of the number of hours of tutoring; it will be weighted by per class priorities (more priority to classes where more hours were requested) and the individual priorities given in the input file for particular student and class combinations. To scale how important the priorities are versus just maximizing the number of hours of tutoring, we'll use two parameters $\lambda_c$ for priorities for entire classes and $\lambda_s$ for priorities for particular students (in particular classes). The function to be maximized is then:
$$\sum_{i,j,k} A_{i,j,k} * (\lambda_c * PC_c) * (\lambda_s * PS_{j,k})$$
where $PC_i$ is the priority for the $i$'th class and $PS_{i,j}$ is the priority of class $j$ for student $i$ (to make things simpler, we can scale all of the priorities by their respective $\lambda$'s ahead of time and drop those terms in the objective function)

**Constraints**
* $A_{i,j,k}=0$ if $T_i$ or $S_j$ doesn't have $C_k$ in their classes lists.
    * We can do this by multiplying by 0 the cells that are valid and the others by 1; then we constrain the sum to be 0. 
* The maximum number of hours per match is 3, so each cell is $\le3$. Each cell must also be $\ge0$, so we can just enter bounds of (0,3) (inclusive) for each variable.
* hours requested/available


**Desired Constraints**

(but impossible to implement in this framework? Perhaps in a quadratic program?)
These can be added post-hoc.
* Each tutor has $\le3$ tutees; this is a little tricky. You need a way to make each cell equal to one, then you can constrain that the sum for each tutor, across all students and classes, is $\le3$. I don't think one can normalize the cells linearly, however.

In [8]:
def get_idx(tutor_idx, tutee_idx, class_idx):
    """
    Computes the index in the 1D array corresponding to (tutor_idx, tutee_idx, class_idx) in the
    imagined 3D tensor
    """
    assert tutor_idx < n_tutors
    assert tutee_idx < n_tutees
    assert class_idx < n_classes
    return tutee_idx + n_tutees * tutor_idx + n_tutees * n_tutors * class_idx

def get_triple_idx(idx):
    """
    Does the inverse of get_idx: returns the (tutor_idx, tutee_idx, class_idx) corresponding to idx
    """
    class_idx = 0
    while idx - (n_tutees * n_tutors) >= 0:
        class_idx += 1
        idx -= (n_tutees * n_tutors)
    
    tutor_idx = 0
    while idx - n_tutees >= 0:
        tutor_idx += 1
        idx -= n_tutees
    tutee_idx = idx
    return tutor_idx, tutee_idx, class_idx

In [9]:
n_variables = n_tutors * n_tutees * n_classes
max_hours = 3
var_bounds = (0, max_hours) # same bounds for all variables

In [10]:
class_list_bound = 0
class_list_constraint = np.ones((1, n_variables)) # set indices to 0 where the proposed matchings are valid

for tutor_idx in xrange(n_tutors):
    tutor_class_indices = get_class_idx([elem[1] for elem in tutor_info.classes.iloc[tutor_idx]]) # elem[1] is class name
    for class_idx in tutor_class_indices:
        for tutee_idx in xrange(n_tutees):
            tutee_class_indices = get_class_idx([elem[1] for elem in tutee_info.classes.iloc[tutee_idx]])
            if class_idx in tutee_class_indices:
                class_list_constraint[0, get_idx(tutor_idx, tutee_idx, class_idx)] = 0

In [11]:
# hours requested/available bounds
# similar to above but tutees need one constraint per class (# hours requested is per class)

hours_constraints = []
hours_bounds = []

for tutor_idx in xrange(n_tutors):
    class_indices = get_class_idx([elem[1] for elem in tutor_info.classes.iloc[tutor_idx]]) # elem[1] is class name
    hours_bounds.append(tutor_info.avail_hours.iloc[tutor_idx])
    constraint = np.zeros((1, n_variables)) # set indices to 1 where the proposed class is valid for this tutor
    for class_idx in class_indices:
        for tutee_idx in xrange(n_tutees):
            constraint[0, get_idx(tutor_idx, tutee_idx, class_idx)] = 1
    hours_constraints.append(constraint)

for tutee_idx in xrange(n_tutees):
    class_indices = get_class_idx([elem[1] for elem in tutee_info.classes.iloc[tutee_idx]]) # elem[1] is class name
    hours_requested = [elem[2] for elem in tutee_info.classes.iloc[tutee_idx]]
    for i in xrange(len(class_indices)):
        class_idx = class_indices[i]
        hours_bounds.append(hours_requested[i])
        constraint = np.zeros((1, n_variables)) # set indices to 1 where the proposed class is valid for this tutee
        for tutor_idx in xrange(n_tutors):
            constraint[0, get_idx(tutor_idx, tutee_idx, class_idx)] = 1
        hours_constraints.append(constraint)

hours_constraints = np.concatenate(hours_constraints, axis=0)
hours_bounds = np.array(hours_bounds)

In [130]:
def get_objective(lambda_classes, lambda_students):
    # scale priorities by lambdas
    scaled_class_priorities = lambda_classes * class_priority.priority.values
    objective_function = np.ones(n_variables)

    for class_idx in xrange(n_classes):
        priority = scaled_class_priorities[class_idx]
        if priority > 1:
            print(priority)
        for tutor_idx in xrange(n_tutors):
            for tutee_idx in xrange(n_tutees):
                objective_function[get_idx(tutor_idx, tutee_idx, class_idx)] *= priority

    for tutee_idx in xrange(n_tutees):
        class_indices = get_class_idx([elem[1] for elem in tutee_info.classes.iloc[tutee_idx]]) # elem[1] is class name
        priorities = [elem[3] for elem in tutee_info.classes.iloc[tutee_idx]]
        for i in xrange(len(class_indices)):
            class_idx = class_indices[i]
            priority = lambda_students * (1 + priorities[i]) # so priority of 0 -> 1
            for tutor_idx in xrange(n_tutors):
                before = objective_function[get_idx(tutor_idx, tutee_idx, class_idx)]
                objective_function[get_idx(tutor_idx, tutee_idx, class_idx)] *= priority
                after = objective_function[get_idx(tutor_idx, tutee_idx, class_idx)]
    return objective_function


def solve(objective_function):
    solution = linprog(-objective_function, options={'disp': True},
                       A_ub=hours_constraints, b_ub=hours_bounds,
                       A_eq=class_list_constraint, b_eq=class_list_bound)
    if solution.x.max() > max_hours:
        print('Quick solution exceeded max_hours ({} hours in a matching; max is {}).'.format(solution.x.max(), max_hours))
        print('Running slower, bounded program.')
        solution = linprog(-objective_function, bounds=(0, max_hours), options={'disp': True},
                           A_ub=hours_constraints, b_ub=hours_bounds,
                           A_eq=class_list_constraint, b_eq=class_list_bound)
    return solution

In [131]:
%%time
objective_function = get_objective(2, 2)
solution = solve(objective_function)

Optimization terminated successfully.
         Current function value: -37.966667  
         Iterations: 47
CPU times: user 94 ms, sys: 1.55 ms, total: 95.5 ms
Wall time: 93.7 ms


In [132]:
solution.x[solution.x > 0]

array([ 3.,  4.,  1.,  1.,  2.,  2.,  2.,  1.,  3.,  2.,  2.,  2.,  1.,
        1.,  2.,  1.,  1.,  1.])

In [78]:
def get_matching(solution, i=0, save=False):
    """
    :param solution: returned from scipy.optimize.linprog
    :param i: number of this matching; used to name the saved file
    """
    matched_indices = np.argwhere(solution.x != 0).ravel()
    matches = []
    for matched_idx in matched_indices:
        tutor_idx, tutee_idx, class_idx = get_triple_idx(matched_idx)
        tutor_id = idx_to_tutor[tutor_idx]
        tutor_name = tutor_info.name.loc[tutor_id]
        tutee_id = idx_to_tutee[tutee_idx]
        tutee_name = tutee_info.name.loc[tutee_id]
        class_name = idx_to_class[class_idx]
        class_id = class_to_id[class_name]
        n_hours = solution.x[matched_idx]
        matches.append([tutor_id, tutor_name, tutee_id, tutee_name, class_id, class_name, n_hours])
    matches = pd.DataFrame(matches,
                           columns=['tutor_id', 'tutor_name', 'tutee_id', 'tutee_name', 'class_id', 'class_name', 'n_hours'])
    if save:
        matches.to_csv(data_path + 'matches_{}.tsv'.format(i), sep='\t', index=False)
    return matches

In [79]:
matches = get_matching(solution)
matches.head()

,tutor_id,tutor_name,tutee_id,tutee_name,class_id,class_name,n_hours
0,3,csi,34,graihjn,2,6.005,3
1,13,esye,28,azusszo,0,6.01,1
2,13,esye,32,iaigstsee,5,6.034,3
3,1,ieaiqs,35,iktwiaaia,4,6.036,1
4,7,xraveukp,35,iktwiaaia,4,6.036,1


In [93]:
lambda_classes = [2, 5]
lambda_students = [2, 3]
use_product = True
if use_product:
    lambdas = product(lambda_classes, lambda_students)
else:
    lambdas = zip(lambda_classes, lambda_students)

for i in xrange(len(lambdas)):
    objective_function = get_objective(*lambdas[i])
    solution = 
    write_matching(solution)

# Matching Characterization

In [4]:
matching = pd.read_csv('matches_1.tsv', sep='\t')
matching.head()

,tutor_id,tutor_name,tutee_id,tutee_name,class_id,class_name,n_hours
0,6,awwugiu,28,ewrmx,9,6.004,1
1,9,iya,28,ewrmx,9,6.004,1
2,15,sqspbra,28,ewrmx,9,6.004,1
3,16,uvyptsnni,25,gosiervyd,9,6.004,2
4,16,uvyptsnni,28,ewrmx,9,6.004,2


In [11]:
matching.tutor_id.value_counts().sort_index()

1     1
2     2
3     1
4     2
6     1
7     1
8     1
9     1
10    2
11    1
13    1
14    1
15    1
16    2
17    1
18    2
Name: tutor_id, dtype: int64

In [9]:
matching.tutor_id.unique().shape

(16,)

In [7]:
pd.read_csv('tutor_info.txt', sep='\t', index_col=0)

,name,classes,avail_hours,n_matches
id,,,,
1,uxi,"[(6, '6.046'), (1, '6.001'), (2, '6.005')]",2,0
2,ythabdbo,"[(2, '6.005'), (6, '6.046')]",3,0
3,vivizvim,"[(7, '6.047'), (13, '6.042')]",1,1
4,opr,"[(3, '6.006'), (6, '6.046'), (8, '6.802'), (11...",3,1
5,krhohuk,"[(5, '6.034'), (1, '6.001')]",2,3
6,awwugiu,"[(4, '6.036'), (9, '6.004')]",1,0
7,jmeumvw,"[(12, '6.041'), (6, '6.046'), (7, '6.047')]",2,0
8,seri,"[(6, '6.046')]",1,1
9,iya,"[(4, '6.036'), (5, '6.034'), (9, '6.004')]",1,0
